# 🌊 Blue-Cloud Marine Analysis Template

This Jupyter notebook template provides Python equivalents to the Blue-Cloud Marine Notebook functions.

## Setup and Installation

First, install the required packages:
```bash
pip install -r python-requirements.txt
```

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gsw  # Gibbs SeaWater package
from geopy.distance import geodesic
from scipy.stats import entropy
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")

print("🌊 Blue-Cloud Marine Analysis Environment Ready!")

## 🔧 Marine Analysis Functions

Python equivalents of the Blue-Cloud notebook functions:

In [ ]:
class MarineAnalyzer:
    """Marine data analysis functions - Python equivalent of Blue-Cloud notebook helpers"""
    
    @staticmethod
    def calculate_density(temperature, salinity, pressure=0):
        """Calculate seawater density using GSW library
        
        Args:
            temperature (float): Temperature in °C
            salinity (float): Practical salinity (PSU)
            pressure (float): Pressure in dbar (default: 0 for surface)
            
        Returns:
            float: Density in kg/m³
        """
        return gsw.rho(salinity, temperature, pressure)
    
    @staticmethod
    def find_thermocline_depth(temperature, depth):
        """Find thermocline depth from temperature gradient
        
        Args:
            temperature (array): Temperature profile
            depth (array): Corresponding depths
            
        Returns:
            float: Thermocline depth in meters
        """
        gradients = np.gradient(temperature, depth)
        thermocline_idx = np.argmin(gradients)  # Most negative gradient
        return depth[thermocline_idx]
    
    @staticmethod
    def calculate_shannon_diversity(species_counts):
        """Calculate Shannon diversity index
        
        Args:
            species_counts (array): Array of species counts
            
        Returns:
            float: Shannon diversity index
        """
        proportions = np.array(species_counts) / np.sum(species_counts)
        proportions = proportions[proportions > 0]  # Remove zeros
        return entropy(proportions, base=np.e)
    
    @staticmethod
    def calculate_distance(coord1, coord2):
        """Calculate distance between two coordinates
        
        Args:
            coord1 (tuple): (latitude, longitude) of first point
            coord2 (tuple): (latitude, longitude) of second point
            
        Returns:
            float: Distance in kilometers
        """
        return geodesic(coord1, coord2).kilometers
    
    @staticmethod
    def generate_ocean_data(max_depth=100, num_points=11):
        """Generate realistic oceanographic profile data
        
        Args:
            max_depth (float): Maximum depth in meters
            num_points (int): Number of data points
            
        Returns:
            dict: Dictionary with temperature, salinity, depth, and coordinates
        """
        depths = np.linspace(0, max_depth, num_points)
        
        # Realistic temperature profile (decreasing with depth)
        surface_temp = 20 + np.random.normal(0, 2)
        temperatures = surface_temp - (depths * 0.1) + np.random.normal(0, 0.5, len(depths))
        
        # Realistic salinity profile (slight increase with depth)
        salinities = 35 + (depths * 0.001) + np.random.normal(0, 0.1, len(depths))
        
        # Random coordinates in North Atlantic
        lat = 45 + np.random.uniform(-5, 10)
        lon = -10 + np.random.uniform(-10, 20)
        
        return {
            'temperature': temperatures,
            'salinity': salinities,
            'depth': depths,
            'coordinates': {'lat': lat, 'lon': lon},
            'timestamp': [datetime.now() - timedelta(hours=i) for i in range(len(depths))]
        }
    
    @staticmethod
    def generate_species_data(num_observations=10):
        """Generate sample species observation data
        
        Args:
            num_observations (int): Number of observations to generate
            
        Returns:
            list: List of species observation dictionaries
        """
        species_list = ['Cod', 'Haddock', 'Herring', 'Mackerel', 'Tuna', 'Sardine', 'Anchovy', 'Sole', 'Plaice']
        observations = []
        
        for i in range(num_observations):
            obs = {
                'species': np.random.choice(species_list),
                'count': np.random.randint(1, 50),
                'coordinates': {
                    'lat': 45 + np.random.uniform(-5, 10),
                    'lon': -10 + np.random.uniform(-10, 20)
                },
                'depth': np.random.uniform(0, 200),
                'timestamp': datetime.now() - timedelta(days=np.random.randint(0, 30))
            }
            observations.append(obs)
        
        return observations
    
    @staticmethod
    def analyze_oil_spill_risk(vessel_data):
        """Analyze vessel trajectory for oil spill risk
        
        Args:
            vessel_data (list): List of vessel tracking dictionaries
            
        Returns:
            dict: Risk assessment with score and factors
        """
        if len(vessel_data) < 2:
            return {'risk_score': 0, 'risk_factors': ['Insufficient data']}
        
        risk_score = 0
        risk_factors = []
        
        # Check for sudden speed changes
        speeds = [point['speed'] for point in vessel_data]
        speed_changes = [abs(speeds[i] - speeds[i-1]) for i in range(1, len(speeds))]
        
        if max(speed_changes) > 5:
            risk_score += 10
            risk_factors.append('Sudden speed change detected')
        
        # Check for erratic course changes
        headings = [point['heading'] for point in vessel_data]
        heading_changes = [abs(headings[i] - headings[i-1]) for i in range(1, len(headings))]
        
        if max(heading_changes) > 45:
            risk_score += 15
            risk_factors.append('Erratic course changes detected')
        
        # Check proximity to sensitive areas (simplified)
        sensitive_areas = [
            {'lat': 60.0, 'lon': -3.0, 'name': 'North Sea Protected Area'},
            {'lat': 43.5, 'lon': -8.0, 'name': 'Bay of Biscay Marine Park'}
        ]
        
        for point in vessel_data:
            for area in sensitive_areas:
                distance = MarineAnalyzer.calculate_distance(
                    (point['coordinates']['lat'], point['coordinates']['lon']),
                    (area['lat'], area['lon'])
                )
                if distance < 50:  # Within 50km
                    risk_score += 20
                    risk_factors.append(f"Near sensitive area: {area['name']}")
                    break
        
        return {
            'risk_score': min(risk_score, 100),
            'risk_factors': list(set(risk_factors))  # Remove duplicates
        }

# Create analyzer instance
analyzer = MarineAnalyzer()
print("✅ Marine analysis functions loaded successfully!")

## 🌊 Example 1: Oceanographic Analysis

In [ ]:
# Generate sample oceanographic data
ocean_data = analyzer.generate_ocean_data(max_depth=100, num_points=21)

print("🌊 Oceanographic Profile Analysis")
print(f"📍 Location: {ocean_data['coordinates']['lat']:.2f}°N, {abs(ocean_data['coordinates']['lon']):.2f}°W")
print(f"🌡️ Temperature range: {min(ocean_data['temperature']):.1f} to {max(ocean_data['temperature']):.1f} °C")
print(f"🧂 Salinity range: {min(ocean_data['salinity']):.2f} to {max(ocean_data['salinity']):.2f} PSU")

# Find thermocline depth
thermocline_depth = analyzer.find_thermocline_depth(ocean_data['temperature'], ocean_data['depth'])
print(f"🌊 Thermocline depth: {thermocline_depth:.1f} meters")

# Calculate densities
surface_density = analyzer.calculate_density(ocean_data['temperature'][0], ocean_data['salinity'][0])
deep_density = analyzer.calculate_density(ocean_data['temperature'][-1], ocean_data['salinity'][-1])
print(f"💧 Surface density: {surface_density:.2f} kg/m³")
print(f"💧 Deep water density: {deep_density:.2f} kg/m³")

In [ ]:
# Plot the oceanographic profile
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 10))

# Temperature profile
ax1.plot(ocean_data['temperature'], ocean_data['depth'], 'b-', linewidth=2, marker='o')
ax1.axhline(y=thermocline_depth, color='r', linestyle='--', alpha=0.7, label=f'Thermocline ({thermocline_depth:.1f}m)')
ax1.set_xlabel('Temperature (°C)')
ax1.set_ylabel('Depth (m)')
ax1.set_title('🌡️ Temperature Profile')
ax1.invert_yaxis()
ax1.grid(True, alpha=0.3)
ax1.legend()

# Salinity profile
ax2.plot(ocean_data['salinity'], ocean_data['depth'], 'g-', linewidth=2, marker='s')
ax2.set_xlabel('Salinity (PSU)')
ax2.set_ylabel('Depth (m)')
ax2.set_title('🧂 Salinity Profile')
ax2.invert_yaxis()
ax2.grid(True, alpha=0.3)

# Density profile
densities = [analyzer.calculate_density(t, s) for t, s in zip(ocean_data['temperature'], ocean_data['salinity'])]
ax3.plot(densities, ocean_data['depth'], 'purple', linewidth=2, marker='^')
ax3.set_xlabel('Density (kg/m³)')
ax3.set_ylabel('Depth (m)')
ax3.set_title('💧 Density Profile')
ax3.invert_yaxis()
ax3.grid(True, alpha=0.3)

# Temperature gradient
gradients = np.gradient(ocean_data['temperature'], ocean_data['depth'])
ax4.plot(gradients, ocean_data['depth'], 'orange', linewidth=2, marker='d')
ax4.axhline(y=thermocline_depth, color='r', linestyle='--', alpha=0.7)
ax4.set_xlabel('Temperature Gradient (°C/m)')
ax4.set_ylabel('Depth (m)')
ax4.set_title('📈 Temperature Gradient')
ax4.invert_yaxis()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🐟 Example 2: Species Diversity Analysis

In [ ]:
# Generate species observation data
species_data = analyzer.generate_species_data(25)

print("🐟 Species Diversity Analysis")
print(f"🔬 Total observations: {len(species_data)}")

# Create species count summary
species_df = pd.DataFrame(species_data)
species_summary = species_df.groupby('species')['count'].sum().sort_values(ascending=False)

print("\n📊 Species Distribution:")
for species, count in species_summary.items():
    print(f"  {species}: {count} individuals")

# Calculate Shannon diversity
diversity = analyzer.calculate_shannon_diversity(species_summary.values)
print(f"\n📈 Shannon Diversity Index: {diversity:.3f}")

# Calculate distances between observations
if len(species_data) >= 2:
    distances = []
    for i in range(len(species_data)-1):
        dist = analyzer.calculate_distance(
            (species_data[i]['coordinates']['lat'], species_data[i]['coordinates']['lon']),
            (species_data[i+1]['coordinates']['lat'], species_data[i+1]['coordinates']['lon'])
        )
        distances.append(dist)
    
    print(f"\n📏 Average distance between observations: {np.mean(distances):.2f} km")
    print(f"📏 Maximum distance between observations: {max(distances):.2f} km")

In [ ]:
# Visualize species data
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Species abundance bar chart
species_summary.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('🐟 Species Abundance')
ax1.set_xlabel('Species')
ax1.set_ylabel('Total Count')
ax1.tick_params(axis='x', rotation=45)

# Species distribution pie chart
ax2.pie(species_summary.values, labels=species_summary.index, autopct='%1.1f%%', startangle=90)
ax2.set_title('🥧 Species Distribution')

# Observation locations map
lats = [obs['coordinates']['lat'] for obs in species_data]
lons = [obs['coordinates']['lon'] for obs in species_data]
counts = [obs['count'] for obs in species_data]

scatter = ax3.scatter(lons, lats, c=counts, s=[c*3 for c in counts], alpha=0.6, cmap='viridis')
ax3.set_xlabel('Longitude')
ax3.set_ylabel('Latitude')
ax3.set_title('🗺️ Observation Locations')
ax3.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax3, label='Count')

# Depth distribution
depths = [obs['depth'] for obs in species_data]
ax4.hist(depths, bins=10, color='lightcoral', alpha=0.7, edgecolor='black')
ax4.set_xlabel('Depth (m)')
ax4.set_ylabel('Frequency')
ax4.set_title('📊 Depth Distribution of Observations')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 🛢️ Example 3: Oil Spill Risk Assessment

In [ ]:
# Generate vessel tracking data with potential risk factors
vessel_track = [
    {'vessel_id': 'TANKER_001', 'coordinates': {'lat': 60.1, 'lon': -3.2}, 'speed': 12, 'heading': 45, 'timestamp': datetime.now()},
    {'vessel_id': 'TANKER_001', 'coordinates': {'lat': 60.15, 'lon': -3.1}, 'speed': 8, 'heading': 90, 'timestamp': datetime.now()},
    {'vessel_id': 'TANKER_001', 'coordinates': {'lat': 60.2, 'lon': -3.0}, 'speed': 15, 'heading': 135, 'timestamp': datetime.now()},
    {'vessel_id': 'TANKER_001', 'coordinates': {'lat': 60.25, 'lon': -2.9}, 'speed': 3, 'heading': 180, 'timestamp': datetime.now()},
    {'vessel_id': 'TANKER_001', 'coordinates': {'lat': 60.3, 'lon': -2.8}, 'speed': 18, 'heading': 225, 'timestamp': datetime.now()}
]

print("🚢 Oil Spill Risk Assessment")
print(f"📊 Analyzing trajectory with {len(vessel_track)} tracking points...")

# Perform risk assessment
risk_assessment = analyzer.analyze_oil_spill_risk(vessel_track)

print(f"\n⚠️ Risk Assessment Results:")
print(f"🎯 Risk Score: {risk_assessment['risk_score']}/100")
print(f"📋 Risk Factors:")
for factor in risk_assessment['risk_factors']:
    print(f"  • {factor}")

# Risk level interpretation
risk_score = risk_assessment['risk_score']
if risk_score > 50:
    risk_level = "🚨 CRITICAL"
    recommendation = "Immediate intervention required!"
elif risk_score > 30:
    risk_level = "⚠️ HIGH"
    recommendation = "Immediate monitoring recommended"
elif risk_score > 15:
    risk_level = "⚡ MODERATE"
    recommendation = "Continue surveillance"
else:
    risk_level = "✅ LOW"
    recommendation = "Normal operations"

print(f"\n{risk_level} RISK: {recommendation}")

In [ ]:
# Visualize vessel trajectory and risk analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Vessel trajectory map
lats = [point['coordinates']['lat'] for point in vessel_track]
lons = [point['coordinates']['lon'] for point in vessel_track]
speeds = [point['speed'] for point in vessel_track]

# Plot trajectory with speed color coding
scatter = ax1.scatter(lons, lats, c=speeds, s=100, cmap='RdYlBu_r', alpha=0.8)
ax1.plot(lons, lats, 'k--', alpha=0.5, linewidth=1)
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('🚢 Vessel Trajectory (colored by speed)')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax1, label='Speed (knots)')

# Add sensitive area markers
sensitive_areas = [
    {'lat': 60.0, 'lon': -3.0, 'name': 'North Sea Protected Area'},
    {'lat': 60.2, 'lon': -2.5, 'name': 'Marine Reserve'}
]
for area in sensitive_areas:
    circle = plt.Circle((area['lon'], area['lat']), 0.3, color='red', alpha=0.3)
    ax1.add_patch(circle)
    ax1.text(area['lon'], area['lat']-0.1, area['name'], ha='center', fontsize=8)

# Speed profile over time
ax2.plot(range(len(speeds)), speeds, 'b-o', linewidth=2, markersize=8)
ax2.set_xlabel('Time Point')
ax2.set_ylabel('Speed (knots)')
ax2.set_title('⚡ Speed Profile')
ax2.grid(True, alpha=0.3)

# Heading changes
headings = [point['heading'] for point in vessel_track]
ax3.plot(range(len(headings)), headings, 'g-s', linewidth=2, markersize=8)
ax3.set_xlabel('Time Point')
ax3.set_ylabel('Heading (degrees)')
ax3.set_title('🧭 Heading Profile')
ax3.grid(True, alpha=0.3)
ax3.set_ylim(0, 360)

# Risk score visualization
risk_categories = ['Speed\nChanges', 'Course\nChanges', 'Proximity\nto Sensitive\nAreas']
risk_values = [10 if 'speed' in ' '.join(risk_assessment['risk_factors']).lower() else 0,
               15 if 'course' in ' '.join(risk_assessment['risk_factors']).lower() else 0,
               20 if 'sensitive' in ' '.join(risk_assessment['risk_factors']).lower() else 0]

colors = ['red' if val > 0 else 'green' for val in risk_values]
bars = ax4.bar(risk_categories, risk_values, color=colors, alpha=0.7)
ax4.set_ylabel('Risk Score Contribution')
ax4.set_title(f'🎯 Risk Factor Breakdown (Total: {risk_assessment["risk_score"]})')
ax4.set_ylim(0, 25)

# Add value labels on bars
for bar, val in zip(bars, risk_values):
    if val > 0:
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                str(val), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 📊 Advanced Analysis: Combined Marine Assessment

In [ ]:
# Comprehensive marine environment assessment
print("🌊 Comprehensive Marine Environment Assessment")
print("=" * 50)

# Generate multiple datasets
ocean_profiles = [analyzer.generate_ocean_data(max_depth=150) for _ in range(5)]
species_observations = analyzer.generate_species_data(50)

# Analyze temperature variations across profiles
surface_temps = [profile['temperature'][0] for profile in ocean_profiles]
thermocline_depths = [analyzer.find_thermocline_depth(profile['temperature'], profile['depth']) 
                     for profile in ocean_profiles]

print(f"🌡️ Surface Temperature Statistics:")
print(f"   Mean: {np.mean(surface_temps):.2f}°C")
print(f"   Std:  {np.std(surface_temps):.2f}°C")
print(f"   Range: {min(surface_temps):.2f} - {max(surface_temps):.2f}°C")

print(f"\n🌊 Thermocline Depth Statistics:")
print(f"   Mean: {np.mean(thermocline_depths):.1f}m")
print(f"   Std:  {np.std(thermocline_depths):.1f}m")
print(f"   Range: {min(thermocline_depths):.1f} - {max(thermocline_depths):.1f}m")

# Species diversity across different areas
species_df = pd.DataFrame(species_observations)
total_diversity = analyzer.calculate_shannon_diversity(
    species_df.groupby('species')['count'].sum().values
)

print(f"\n🐟 Biodiversity Assessment:")
print(f"   Total species observed: {species_df['species'].nunique()}")
print(f"   Total individuals: {species_df['count'].sum()}")
print(f"   Shannon Diversity Index: {total_diversity:.3f}")

# Environmental quality index (simplified)
temp_score = 100 - abs(np.mean(surface_temps) - 18) * 5  # Optimal around 18°C
diversity_score = min(total_diversity * 30, 100)  # Scale diversity
depth_variability = np.std(thermocline_depths)
stability_score = max(100 - depth_variability * 2, 0)  # Lower variability = more stable

environmental_index = (temp_score + diversity_score + stability_score) / 3

print(f"\n🌍 Environmental Quality Index: {environmental_index:.1f}/100")
print(f"   Temperature Score: {temp_score:.1f}/100")
print(f"   Biodiversity Score: {diversity_score:.1f}/100")
print(f"   Stability Score: {stability_score:.1f}/100")

if environmental_index > 80:
    status = "🟢 EXCELLENT"
elif environmental_index > 60:
    status = "🟡 GOOD"
elif environmental_index > 40:
    status = "🟠 MODERATE"
else:
    status = "🔴 POOR"

print(f"\n{status} - Marine Environment Status")

## 💾 Export Results

Save your analysis results for further use:

In [ ]:
# Create summary report
report = {
    'analysis_date': datetime.now().isoformat(),
    'oceanographic_data': {
        'profiles_analyzed': len(ocean_profiles),
        'surface_temperature_mean': float(np.mean(surface_temps)),
        'surface_temperature_std': float(np.std(surface_temps)),
        'thermocline_depth_mean': float(np.mean(thermocline_depths)),
        'thermocline_depth_std': float(np.std(thermocline_depths))
    },
    'biodiversity_data': {
        'total_observations': len(species_observations),
        'species_count': int(species_df['species'].nunique()),
        'total_individuals': int(species_df['count'].sum()),
        'shannon_diversity': float(total_diversity)
    },
    'environmental_assessment': {
        'overall_index': float(environmental_index),
        'temperature_score': float(temp_score),
        'biodiversity_score': float(diversity_score),
        'stability_score': float(stability_score),
        'status': status
    }
}

# Save to JSON file
import json
with open('marine_analysis_report.json', 'w') as f:
    json.dump(report, f, indent=2)

# Save species data to CSV
species_df.to_csv('species_observations.csv', index=False)

# Save oceanographic data
ocean_summary = pd.DataFrame({
    'profile_id': range(len(ocean_profiles)),
    'surface_temperature': surface_temps,
    'thermocline_depth': thermocline_depths,
    'latitude': [p['coordinates']['lat'] for p in ocean_profiles],
    'longitude': [p['coordinates']['lon'] for p in ocean_profiles]
})
ocean_summary.to_csv('oceanographic_summary.csv', index=False)

print("📁 Analysis results saved:")
print("   • marine_analysis_report.json")
print("   • species_observations.csv")
print("   • oceanographic_summary.csv")
print("\n✅ Analysis complete! Results ready for further processing or sharing.")

## 🔗 Next Steps

This template provides the foundation for marine data analysis in Python. You can extend it by:

1. **Connecting to real data sources** (Copernicus Marine, EMODnet, etc.)
2. **Adding more sophisticated analysis methods**
3. **Creating interactive visualizations** with Plotly or Bokeh
4. **Implementing machine learning models** for prediction
5. **Building automated reporting systems**

For more information, visit the [Blue-Cloud Marine Notebook documentation](README-NOTEBOOK.md).